In [2]:
# fruits data from https://www.kaggle.com/moltean/fruits/data
# food-images-food-101.zip from kaggle too!
# not for first use!

In [3]:
!ls

data  invalid-log  log-1.txt  look on data!.ipynb  models  models.ipynb


In [1]:
src = [ 
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n14974264', 'paper'),
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n03580615', 'net'),
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n09411189', 'forest'),
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n01318894', 'pets'),
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n07687469', 'bread'),
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n07874063', 'pizza'),
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n05257967', 'sausage'),
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n03405725', 'furniture'),
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n07942152', 'people'),
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n00021265', 'food'),
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n07753592', 'banana'),
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n12353203', 'edible banana'),
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n07690019', 'frankfurter'),
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n07865105', 'chili-dog'),
            ('http://image-net.org/api/text/imagenet.synset.geturls?wnid=n07697537', 'hotdog'),
]

In [6]:
import urllib
import urllib.request
import itertools
import os
import cv2

In [7]:
from tqdm import tqdm_notebook
from miniutils.progress_bar import iparallel_progbar
import numpy as np
import time

In [38]:
pic_num = 1

In [7]:
import socket

timeout = 3617
socket.setdefaulttimeout(timeout)

def loadImage(path, url, counter):
    #time.sleep(np.random.randint(low=1, high=10))
    global pic_num
    if pic_num < counter:
        pic_num = counter + 1
    s = ''
    try:                
        urllib.request.urlretrieve(url, path + "/" + str(counter) + ".jpg")
        img = cv2.imread(path + "/" + str(counter) + ".jpg")             
        if img is not None:
            cv2.imwrite(path + "/" + str(counter) + ".jpg", img)
            s = url + " " + str(counter)
    except Exception as e:
        s = "exception = " + str(e)
    return s

In [8]:
def storeImages(src, logfilename):
    s = ""
    global pic_num
    with open(logfilename, 'w') as out:
        for url, path in tqdm_notebook(src):
            s += path + ":\n"
            path = "./data/" + path
            if not os.path.exists(path):
                os.makedirs(path)
            try:
                image_urls = str(urllib.request.urlopen(url).read())
                for x in iparallel_progbar(loadImage, 
                                      zip(itertools.repeat(path), 
                                          image_urls.split('\\n'), 
                                          itertools.count(pic_num)), 
                                      starmap=True,
                                      nprocs=13):
                    s += "Result: " + str(x) + "\n"
            except Exception as e:
                s += "\n$$$\n" + str(e) + "\n$$$\n" 
            s += "###" * 42
        out.write(s)

In [11]:
def removeInvalidImages(dirPaths, logname):
    with open(logname, 'w') as out:
        s = ''
        for dirPath in tqdm_notebook(dirPaths):
            dirPath1 = './data/' + dirPath
            for img in tqdm_notebook(os.listdir(dirPath1)):
                for invalid in os.listdir('./data/invalid'):
                    try:
                        current_image_path = str(dirPath1) + '/' + str(img)
                        invalid = cv2.imread('./data/invalid/' + str(invalid))
                        question = cv2.imread(current_image_path)                     
                        if question is None or invalid.shape == question.shape and not(np.bitwise_xor(invalid, question).any()):
                            s += current_image_path + "\n"
                            os.remove(current_image_path)
                            break
                    except Exception as e:
                        s += str(e) + "\n"
        out.write(s)

In [10]:
storeImages(src, 'log-1.txt')

In [13]:
removeInvalidImages([x[1] for x in src], 'invalid-log')

Then I move images from other folders(food and sausage) which might look like hotdog and banana to appropriate packages.

In [14]:
#set([x[1] for x in src]) ^ {'edible banana', 'banana', 'hotdog', 'frankfurter', 'chili-dog'}

# Let's make a dataset!

In [11]:
paths_for_data = {
    'banana': {'edible banana', 'banana'},
    'hotdog': set(('hotdog', 'frankfurter', 'chili-dog')),
    'nothotdog_notbanana': set(('bread', \
                             'food', \
                             'forest', \
                             'furniture', \
                             'net', \
                             'paper', \
                             'people', \
                             'pets', \
                             'pizza', \
                             'sausage'))
}

data_folder = './to_cnn'

In [12]:
def path_for_label(x):
    for k, v in paths_for_data.items():
        if x in v:
            return k
    return 'nothotdog_notbanana'

In [13]:
import shutil

pic_number = 0

def makeDatasetsForTrain(data_dir, dirPaths, to, sub=False):
    global pic_number
    for dirPath in tqdm_notebook(dirPaths):
        dirPath1 = dirPath
        if not sub:
            dirPath1 = './' + data_dir + "/" + dirPath
            path = data_folder + "/" + to + "/" + path_for_label(dirPath)
        if sub:
            #print("$$$")
            #print(dirPath.split('/'))
            path = data_folder + "/" + to + "/" + path_for_label(dirPath.split('/')[-2])
        
        if not os.path.exists(path):
                os.makedirs(path)
        
        for img in tqdm_notebook(os.listdir(dirPath1)):
            try:
                abs_path = dirPath1 + "/" + img
                if os.path.isdir(abs_path):
                    #print('dir')
                    makeDatasetsForTrain([abs_path], True)
                    #print('end\n')
                else:
                    current_image_path = str(dirPath1) + "/" + str(img)
                    future_image_path = path + "/" + str(pic_number) + '.' + str(img).split('.')[-1]
                    #print(current_image_path)
                    #print(future_image_path)
                    #print()
                    shutil.copyfile(current_image_path, future_image_path)
                    pic_number += 1
            except Exception as e:
                print(e)

# Augmentation of small classes

In [19]:
import Augmentor

p = Augmentor.Pipeline('./data/sausage')

p.shear(probability=0.45, max_shear_left=15, max_shear_right=15)
p.random_distortion(probability=0.75, grid_width=5, grid_height=3, magnitude=10)
p.rotate(probability=0.7, max_left_rotation=15, max_right_rotation=15)
p.zoom(probability=0.5, min_factor=1.0, max_factor=1.5)
p.flip_left_right(probability=0.8)
p.random_brightness(probability=0.7, min_factor=0.5, max_factor=1.0)
p.random_distortion(probability=0.55, grid_width=4, grid_height=5, magnitude=6)
p.crop_random(probability=0.6, percentage_area=0.8)
p.resize(probability=0.3, height=120, width=120)

p.sample(1000)

Executing Pipeline:   0%|          | 0/1000 [00:00<?, ? Samples/s]

Initialised with 6 image(s) found.
Output directory set to ./data/sausage/output.

Processing <PIL.Image.Image image mode=RGB size=120x120 at 0x7FA3B4132630>: 100%|██████████| 1000/1000 [00:17<00:00, 58.74 Samples/s]


In [20]:
p = Augmentor.Pipeline('./data/bread')

p.shear(probability=0.45, max_shear_left=15, max_shear_right=15)
p.random_distortion(probability=0.75, grid_width=5, grid_height=3, magnitude=10)
p.rotate(probability=0.7, max_left_rotation=15, max_right_rotation=15)
p.zoom(probability=0.5, min_factor=1.0, max_factor=1.5)
p.flip_left_right(probability=0.8)
p.random_brightness(probability=0.7, min_factor=0.5, max_factor=1.0)
p.random_distortion(probability=0.55, grid_width=4, grid_height=5, magnitude=6)
p.crop_random(probability=0.6, percentage_area=0.8)
p.resize(probability=0.3, height=120, width=120)

p.sample(500)

Executing Pipeline:   0%|          | 0/500 [00:00<?, ? Samples/s]

Initialised with 744 image(s) found.
Output directory set to ./data/bread/output.

Processing <PIL.Image.Image image mode=RGB size=375x500 at 0x7FA3B4145438>: 100%|██████████| 500/500 [00:13<00:00, 178.07 Samples/s]


In [5]:
# augment target classes

import Augmentor

p = Augmentor.Pipeline('./data/hotdog')

p.shear(probability=0.45, max_shear_left=15, max_shear_right=15)
p.random_distortion(probability=0.75, grid_width=5, grid_height=3, magnitude=10)
p.rotate(probability=0.7, max_left_rotation=15, max_right_rotation=15)
p.zoom(probability=0.5, min_factor=1.0, max_factor=1.5)
p.flip_left_right(probability=0.8)
p.random_brightness(probability=0.7, min_factor=0.5, max_factor=1.0)
p.random_distortion(probability=0.55, grid_width=4, grid_height=5, magnitude=6)
p.crop_random(probability=0.6, percentage_area=0.8)
p.resize(probability=0.3, height=120, width=120)

p.sample(1000)

p = Augmentor.Pipeline('./data/frankfurter')

p.shear(probability=0.45, max_shear_left=15, max_shear_right=15)
p.random_distortion(probability=0.75, grid_width=5, grid_height=3, magnitude=10)
p.rotate(probability=0.7, max_left_rotation=15, max_right_rotation=15)
p.zoom(probability=0.5, min_factor=1.0, max_factor=1.5)
p.flip_left_right(probability=0.8)
p.random_brightness(probability=0.7, min_factor=0.5, max_factor=1.0)
p.random_distortion(probability=0.55, grid_width=4, grid_height=5, magnitude=6)
p.crop_random(probability=0.6, percentage_area=0.8)
p.resize(probability=0.3, height=120, width=120)

p.sample(1500)

p = Augmentor.Pipeline('./data/chili-dog')

p.shear(probability=0.45, max_shear_left=15, max_shear_right=15)
p.random_distortion(probability=0.75, grid_width=5, grid_height=3, magnitude=10)
p.rotate(probability=0.7, max_left_rotation=15, max_right_rotation=15)
p.zoom(probability=0.5, min_factor=1.0, max_factor=1.5)
p.flip_left_right(probability=0.8)
p.random_brightness(probability=0.7, min_factor=0.5, max_factor=1.0)
p.random_distortion(probability=0.55, grid_width=4, grid_height=5, magnitude=6)
p.crop_random(probability=0.6, percentage_area=0.8)
p.resize(probability=0.3, height=120, width=120)

p.sample(1250)

p = Augmentor.Pipeline('./data/banana')

p.shear(probability=0.45, max_shear_left=15, max_shear_right=15)
p.random_distortion(probability=0.75, grid_width=5, grid_height=3, magnitude=10)
p.rotate(probability=0.7, max_left_rotation=15, max_right_rotation=15)
p.zoom(probability=0.5, min_factor=1.0, max_factor=1.5)
p.flip_left_right(probability=0.8)
p.random_brightness(probability=0.7, min_factor=0.5, max_factor=1.0)
p.random_distortion(probability=0.55, grid_width=4, grid_height=5, magnitude=6)
p.crop_random(probability=0.6, percentage_area=0.8)
p.resize(probability=0.3, height=120, width=120)

p.sample(2300)


p = Augmentor.Pipeline('./data/edible banana')

p.shear(probability=0.45, max_shear_left=15, max_shear_right=15)
p.random_distortion(probability=0.75, grid_width=5, grid_height=3, magnitude=10)
p.rotate(probability=0.7, max_left_rotation=15, max_right_rotation=15)
p.zoom(probability=0.5, min_factor=1.0, max_factor=1.5)
p.flip_left_right(probability=0.8)
p.random_brightness(probability=0.7, min_factor=0.5, max_factor=1.0)
p.random_distortion(probability=0.55, grid_width=4, grid_height=5, magnitude=6)
p.crop_random(probability=0.6, percentage_area=0.8)
p.resize(probability=0.3, height=120, width=120)

p.sample(1000)

Executing Pipeline:   0%|          | 0/1000 [00:00<?, ? Samples/s]

Initialised with 705 image(s) found.
Output directory set to ./data/hotdog/output.

Processing <PIL.Image.Image image mode=RGB size=280x209 at 0x7FC9347BC978>: 100%|██████████| 1000/1000 [00:16<00:00, 59.13 Samples/s]
Executing Pipeline:   0%|          | 0/1500 [00:00<?, ? Samples/s]

Initialised with 497 image(s) found.
Output directory set to ./data/frankfurter/output.

Processing <PIL.Image.Image image mode=RGB size=500x333 at 0x7FC93C0AE4A8>: 100%|██████████| 1500/1500 [00:27<00:00, 55.13 Samples/s] 
Executing Pipeline:   0%|          | 0/1250 [00:00<?, ? Samples/s]

...

Processing <PIL.Image.Image image mode=RGB size=400x300 at 0x7FC93C482A58>: 100%|██████████| 1250/1250 [00:28<00:00, 43.59 Samples/s]   
Executing Pipeline:   0%|          | 0/2300 [00:00<?, ? Samples/s]

Initialised with 1089 image(s) found.
Output directory set to ./data/banana/output.

Processing <PIL.Image.Image image mode=RGB size=120x120 at 0x7FC93C3F3DA0>: 100%|██████████| 2300/2300 [00:46<00:00, 49.68 Samples/s] 
Executing Pipeline:   0%|          | 0/1000 [00:00<?, ? Samples/s]

Initialised with 95 image(s) found.
Output directory set to ./data/edible banana/output.

Processing <PIL.Image.Image image mode=RGB size=300x400 at 0x7FC9347DE208>: 100%|██████████| 1000/1000 [00:22<00:00, 43.77 Samples/s]


In [3]:
def split_dataset_into_test_and_train_sets(data_dir, train_dir, val_dir, test_dir, split, seed=42):
    import shutil
    import numpy as np
    import os
    from tqdm import tqdm_notebook
    
    np.random.seed(seed)
    
    # Recreate testing and training directories
    
    for x in [train_dir, val_dir, test_dir]:
        if not os.path.exists(x):
                os.makedirs(x)
        if test_dir.count('/') > 1:
            shutil.rmtree(x, ignore_errors=False)
            os.makedirs(x)
            print("Successfully cleaned directory " + x + " !")
        else:
            print("Refusing to delete data directory " + x + " as we prevent you from doing stupid things!")
    
        
    num_train_files = 0
    num_test_files = 0
    num_val_files = 0

    for subdir, dirs, files in os.walk(data_dir):
        category_name = os.path.basename(subdir)

        if category_name == 'output':
            #print(os.path.abspath(subdir))
            category_name = os.path.abspath(subdir).split('/')[-2]
            #print(category_name)
        
        # Don't create a subdirectory for the root directory
        #print(category_name + " vs " + os.path.basename(data_dir))
        if category_name == os.path.basename(data_dir) or category_name == 'invalid':
            continue

        train_category_dir = train_dir + '/' + category_name
        test_category_dir = test_dir + '/' + category_name
        val_category_dir = val_dir + '/' + category_name

        for x in [train_category_dir, val_category_dir, test_category_dir]:
            if not os.path.exists(x):
                os.mkdir(x)
                
        for file in tqdm_notebook(files):
            input_file = os.path.join(subdir, file)
            r = np.random.rand()
            if r < split:
                shutil.copy(input_file, test_category_dir + "/" + file)
                num_test_files += 1
            elif r >= split and r < 2 * split:
                shutil.copy(input_file, val_category_dir + "/" + file)
                num_val_files += 1
            else:
                shutil.copy(input_file, train_category_dir + "/" + file)
                num_train_files += 1

    print("Processed " + str(num_train_files) + " training files.")
    print("Processed " + str(num_test_files) + " testing files.")
    print("Processed " + str(num_val_files) + " validation files.")

In [4]:
split_dataset_into_test_and_train_sets('./data/', './train/', './val/', './test/', 0.1)

Successfully cleaned directory ./train/ !
Successfully cleaned directory ./val/ !
Successfully cleaned directory ./test/ !



Processed 14915 training files.
Processed 1892 testing files.
Processed 1856 validation files.


In [14]:
# make new categories!
from tqdm import tqdm
for xx in tqdm(['train', 'val', 'test']):
    print(xx, "!")
    makeDatasetsForTrain(xx, [x[1] for x in src], xx)

  0%|          | 0/3 [00:00<?, ?it/s]

train !


 33%|███▎      | 1/3 [00:02<00:05,  2.73s/it]


val !


 67%|██████▋   | 2/3 [00:03<00:02,  2.15s/it]


test !


100%|██████████| 3/3 [00:04<00:00,  1.77s/it]

In [15]:
import os

def number_of_files(path):
    return len([name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))])

In [17]:
t = []
for y in ['train', 'val', 'test']:
    tmp = []
    for x in ['banana', 'hotdog', 'nothotdog_notbanana']:
        tmp.append(number_of_files("./to_cnn/" + y + "/" + x))
    print(tmp)
    t.append(sum(tmp))

[3608, 4506, 6801]
[442, 539, 875]
[433, 548, 911]


In [18]:
list(map(lambda x: x / sum(t), t))

[0.7991748379145903, 0.09944810587794031, 0.10137705620746933]

That's all folks!